In [2]:
import numpy as np
import csv
import math
from scipy import integrate
from scipy.integrate import dblquad
from scipy.integrate import nquad

with open("./output.csv","r") as file:
    reader = csv.reader(file)
    data = [row for row in reader]
    del data[0]

    
data = np.array([[np.fromstring(row[0][1:-1], sep=' '), np.fromstring(row[1][1:-1], sep=' ')] for row in data], dtype=np.ndarray)

n = len(data)
# array1: init
# array2: next
init_data = data[:,0].astype(float)
next_data = data[:,1].astype(float)


init_data[:5]




array([[29.03826179,  3.88696108,  0.93189377, 14.69331688,  0.40503613,
         0.30920791,  1.21971565],
       [54.95302721,  2.02727464,  0.85127559, 19.40564531,  1.00521496,
         1.89308441,  1.26134618],
       [ 8.08699657,  1.67134396,  0.83478328, 17.83689669,  0.59947075,
         0.78829184,  1.18719324],
       [84.89488312,  3.45786336,  0.84952332, 18.45729785,  0.84550258,
         0.35813699,  1.33523856],
       [27.55769618,  5.47663666,  0.87258155, 21.96307525,  0.88351588,
         0.64035527,  1.33186086]])

In [3]:
# calc std
std_init = np.std(init_data,axis=0,ddof=1)
# std_total_init = np.std(init_data,axis=0)

std_next = np.std(next_data,axis=0,ddof=1)

std_init,std_next

(array([23.79891679,  1.69062488,  0.11783135,  2.67007887,  0.30065055,
         0.46408763,  0.06030478]),
 array([23.75357656,  1.71972549,  0.11783135,  2.67007886,  0.30046535,
         0.70858364,  0.03825582]))

$h_{init} = \left(\frac{4}{3n}\right)^{\frac{1}{5}}\cdot std_{init}$

## need to check

$h = Cn^{-\frac{1}{d+4}}(\det(\Sigma))^{\frac{1}{2}}$ 
in book

$h = Cn^{-\frac{1}{d+4}}(\det(\Sigma))^{\frac{1}{d+4}}$

In [21]:
# scott's rule
# Nonparametric and Semiparametric Models, by Wolfgang p104
h_init = (4/3/n)**(1/5)*std_init

h_next = (4/3/n)**(1/5)*std_next

h_init, h_next

(array([10.03563686,  0.71291049,  0.04968767,  1.12593116,  0.12677971,
         0.19569861,  0.0254296 ]),
 array([10.01651758,  0.72518177,  0.04968767,  1.12593115,  0.12670161,
         0.29879881,  0.01613189]))

In [26]:
xf = 1/((86-0)*(6-0)*(0.18+0.18)*(21-12)*(0.5+0.5)*(0.8+0.8)*(0.1+0.1))
init_data_transposed = init_data.transpose()
next_data_transposed = next_data.transpose()

In [27]:
# kerdenfunction

def kerdenfunction(x_point, y_point, x, xf, y, hx_cv, hy_cv):
    d = x.shape[0] # Generate the dimension of our study problem, x should be dxn
    n = x.shape[1]
    # print(d,n)
    
    f1 = 0
    for i_ker in range(n):
        f1 = f1 + ((n * np.prod(hx_cv) * np.prod(hy_cv))**(-1) * (2 * np.pi)**(-0.5*d) *
             np.prod(np.exp(-((x_point - x[:, i_ker]) / hx_cv)**2 / 2)) *
             np.prod(np.exp(-((y_point - y[:, i_ker]) / hy_cv)**2 / 2)))
    store = f1 / xf
    return store


In [28]:

kerdenfunction([1,2,0,13,0,0,0],[1,2,0,13,0,0,0],init_data_transposed, xf, next_data_transposed,h_init,h_next)

0.0

In [ ]:
import numpy as np
from scipy.integrate import dblquad
from itertools import product

def prob_density(x, y):
    return kerdenfunction(x,y,init_data_transposed,xf,next_data_transposed,h_init,h_next)   

# Define the number of divisions in each axis
nx = 10
ny = 10

# Define the range of x and y
x_min, x_max = 0, 86
y_min, y_max = 0, 6



# Calculate the range of each subdivision
x_range = np.linspace(x_min, x_max, nx+1)
y_range = np.linspace(y_min, y_max, ny+1)

# Store the probability value for each small area
prob_matrix = np.zeros((nx, ny, nx, ny))

# Iterate over all small regions and calculate the probability of the probability density function within the small region
for i, j, k, l in product(range(nx), range(ny), range(nx), range(ny)):
    # Calculate the range of values for the current small area
    curr_x_range = (x_range[i], x_range[i+1])
    curr_y_range = (y_range[j], y_range[j+1])
    curr_xf = (curr_x_range[1] - curr_x_range[0]) * (curr_y_range[1] - curr_y_range[0])
    
    # Calculate the Le Berger integral
    prob_matrix[i, j, k, l] = dblquad(prob_density, curr_x_range[0], curr_x_range[1],
                                       lambda x: curr_y_range[0], lambda x: curr_y_range[1])[0] / curr_xf

# Print results
print(prob_matrix)


In [29]:
import numpy as np
from scipy.integrate import nquad
from itertools import product

def prob_density(x,y):
    return kerdenfunction(x, y, init_data_transposed, xf, next_data_transposed, h_init, h_next)

# Define the number of divisions in each axis for the 7 arrays
nx = [10]*7
ny = [10]*7

# Define the range of values for each axis of the 7 arrays
x_min = [0]*7
x_max = [86]*7

y_min = [0]*7
y_max = [6]*7

# Calculate the range of each subdivision for each axis of the 7 arrays
x_range = [np.linspace(x_min[i], x_max[i], nx[i]+1) for i in range(7)]
y_range = [np.linspace(y_min[i], y_max[i], ny[i]+1) for i in range(7)]

# Store the probability value for each small area
prob_matrix = np.zeros(tuple(nx + ny))

# Iterate over all small regions and calculate the probability of the probability density function within the small region
for i, j, k, l, m, n, o in product(*[range(nx[i]) for i in range(7)]):
    # Calculate the range of values for the current small area for each axis of the 7 arrays
    curr_x_range = tuple((x_range[i][i_val], x_range[i][i_val+1]) for i, i_val in enumerate((i, j, k, l, m, n, o)))
    curr_y_range = tuple((y_range[i][i_val], y_range[i][i_val+1]) for i, i_val in enumerate((i, j, k, l, m, n, o)))
    curr_xf = np.prod([curr_x_range[i][1] - curr_x_range[i][0] for i in range(7)])
    
    # Calculate the n-dimensional Lebesgue integral
    prob_matrix[i, j, k, l, m, n, o] = nquad(prob_density, [curr_x_range[i] for i in range(7)] + [curr_y_range[i] for i in range(7)])[0] / curr_xf

# Print results
print(prob_matrix)


MemoryError: Unable to allocate 728. TiB for an array with shape (10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10) and data type float64

In [171]:
def f(x1,y1,x2,y2,a,b,c):
    print("yep")
    print(x1,y1,x2,y2,a,b,c)
    return a+b+c

x1_min, x1_max = 0, 1
x2_min, x2_max = 0, 1
y1_min, y1_max = 0, 1
y2_min, y2_max = 0, 1



a = b = c = 1


# args_order = (0, 1, 2, 3)

# result, error = nquad(f, [(x1_min, x1_max), (x2_min, x2_max), (y1_min, y1_max), (y2_min, y2_max)],args= (a,b,c,))
                      

# print(result)



In [118]:
# cal max and min pro

# x:[0,84]
# y:[0,6]
i_step = 4
j_step = 2
def calc():
    
        for i in range(0,84,i_step):
            for j in range(0,6,j_step):
                # now
                # x:[i,i+i_step] y:[j,j+j_step]
                
                # to
                # other all
                
                
                x1_min, x1_max = i, i+i_step
                x2_min, x2_max = j, j+j_step
                
                y1_min, y1_max = 0, 84
                y2_min, y2_max = 0, 6
                
                args_order = (0, 1, 2, 3)
                
                x_point = [0,0,0,0,0,0,0]
                y_point = [0,0,0,0,0,0,0]
                # kerdenfunction(x_point,y_point, init_data_transposed, xf, next_data_transposed,h_init,h_next)
                
                result, error = nquad(kerdenfunction,  [(x1_min, x1_max), (x2_min, x2_max), (y1_min, y1_max), (y2_min, y2_max)],
                      args=(arr1[0], arr1[1], arr2[0], arr2[1], init_data_tra)

SyntaxError: invalid syntax (2508729293.py, line 8)

### Compare with real data

In [1]:
tau = 0.001
# gauss_noise_matrix = gauss_noise(np.zeros((n+1,7,1)),1,0.2)
delta = 0.2
        
# def delta(x):
#     if x == 0:
#         return np.inf
#     else:
#         return 0

In [6]:
# len(init_data)
init_data[0]

array([29.03826179,  3.88696108,  0.93189377, 14.69331688,  0.40503613,
        0.30920791,  1.21971565])

In [8]:
# x1
# < 0.1
# a1 = x4(k)cos(x5(k))
# b1 = x4(k)cos(x5(k)+x7(k))

# x: [,] 7-dim
# x1:x[0]

def y1(x,x_next):
    if x[0]<0.1:
        # f = math.sqrt(2)/(delta*math.sqrt(math.pi))*(math.e**(-2*((x_next[0]-x[0]-tau*a1)/delta)**2
        f = math.sqrt(2)/(delta*math.sqrt(math.pi))*(math.e**(-2*((x_next[0]-x[0]-tau*x[3]*math.cos(x[4]))/delta)**2))
    else:
        TODO! later all not finished!!!
        # f = math.sqrt(2)/(delta*math.sqrt(math.pi))*(math.e**(-2*((x_next[0]-x[0]-tau*b1)/delta)**2
        f = math.sqrt(2)/(delta*math.sqrt(math.pi))*(math.e**(-2*((x_next[0]-x[0]-tau*b1**2))/delta**2))
        
for i in range(len(init_data)-1):
    y1(init_data[i],init_data[i+1])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# x2
# < 0.1
def y2(x,x_next):
    if x[1]<0.1:
        # f = math.sqrt(2)/(delta*math.sqrt(math.pi))*(math.e**(-2*((x_next[1]-x[1]-tau*a2)/delta)**2
        f = math.sqrt(2)/(delta*math.sqrt(math.pi))*(math.e**(-2*((x_next[1]-x[1]-tau*!!/delta)**2))
    else:
        TODO! later all not finished!!!
        # f = math.sqrt(2)/(delta*math.sqrt(math.pi))*(math.e**(-2*((x_next[1]-x[1]-tau*b2)/delta)**2
        f = math.sqrt(2)/(delta*math.sqrt(math.pi))*(math.e**(-2*((x_next[]-x[1]-tau*b2**2))/delta**2))

In [ ]:
# x3
def y3(x,x_next):
    # f = math.sqrt(2)/(delta*math.sqrt(math.pi))*(math.e**(-2*((x_next[2]-x[2]-tau*Sat1(V1))/delta)**2

In [ ]:
# x4
def y4(x,x_next):
    # f = math.sqrt(2)/(delta*math.sqrt(math.pi))*(math.e**(-2*((x_next[3]-x[3]-tau*Sat2(V2))/delta)**2

In [ ]:
# x5
# < 0.1
def y5(x,x_next):
    if x[4]<0.1:
        # f = math.sqrt(2)/(delta*math.sqrt(math.pi))*(math.e**(-2*((x_next[4]-x[4]-tau*a5)/delta)**2
    else:
        TODO! later all not finished!!!
        # f = math.sqrt(2)/(delta*math.sqrt(math.pi))*(math.e**(-2*((x_next[4]-x[4]-tau*b5)/delta)**2

In [ ]:
# x6
# < 0.1
def y6(x,x_next):
    if x[5]<0.1:
        # f = math.sqrt(2)/(delta*math.sqrt(math.pi))*(math.e**(-2*((x_next[5]-x[5]-tau*a6)/delta)**2
    else:
        # f = math.sqrt(2)/(delta*math.sqrt(math.pi))*(math.e**(-2*((x_next[5]-x[5]-tau*b6)/delta)**2

In [ ]:
# x7
# < 0.1
def y7(x,x_next):
    if x[6]<0.1:
        # f = math.sqrt(2)/(delta*math.sqrt(math.pi))*(math.e**(-2*((x_next[6]-x[6]-tau*a7)/delta)**2
    else:
        # f = math.sqrt(2)/(delta*math.sqrt(math.pi))*(math.e**(-2*((x_next[6]-x[6]-tau*b7)/delta)**2